## F1 state representation quality using multiple states  

In [1]:
from bbvi_infer_2l import * 
from f1 import *
%matplotlib inline
import pickle
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import silhouette_score

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Read in data observations

In [2]:
year= 2024
gp = 'China'
event = 'Race'

session_event = ff1.get_session(year, gp, event)
session_event.load()
circuit_info = session_event.get_circuit_info()

req         WARNING 	DEFAULT CACHE ENABLED! (396.53 MB) /root/.cache/fastf1
core           INFO 	Loading data for Chinese Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 1 completed the race distance 00:08.313000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 d

In [3]:
# Train data
pos_winner= generate_full_laps(session_event.laps.pick_driver('VER'), 'VER')
## States 
states_train, zs_train = one_hot_states_multilap(pos_winner[['X','Y']].to_numpy(), circuit_info.corners[['X','Y']].to_numpy(), 
                                    pos_winner['LapNumber'].to_numpy())

/opt/conda/lib/python3.10/site-packages/fastf1/core.py:3183: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"


In [4]:
## Standardize train data
import numpy.random as npr
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
track_norm = scaler.fit_transform(pos_winner[['X','Y']].to_numpy())
C = npr.randn(10, 2)  ## emission matrix
ys_train = (C @ track_norm.T).T + npr.randn(track_norm.shape[0], 10)*0.01

In [5]:
## Test data
pos_alt= generate_full_laps(session_event.laps.pick_driver('SAI'), 'SAI')
### States
states_test, zs_test = one_hot_states_multilap(pos_alt[['X','Y']].to_numpy(), circuit_info.corners[['X','Y']].to_numpy(), 
                                               pos_alt['LapNumber'].to_numpy())
### Observations
track_norm = scaler.transform(pos_alt[['X','Y']].to_numpy())
ys_test = (C @ track_norm.T).T + npr.randn(track_norm.shape[0], 10)*0.01

/opt/conda/lib/python3.10/site-packages/fastf1/core.py:3183: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"


## GSD

In [6]:
gsd = []
for seed in range(3):  ## repeat tests
    torch.manual_seed(seed)  # Set PyTorch seed
    torch.cuda.manual_seed_all(seed)  # Set CUDA seed 
    accs = []
    for K in range(2, 11):
        model = GenerativeSLDS(N=10, K=K, D=2, emission_model="gaussian")
        elbos, variational_z = fit_bbvi_schedule(model.to('cuda'), ys = torch.tensor(ys_train).to('cuda').float(), 
                                             num_iters=4000, learning=True, n_samples=10, base_lr=1e-2, warmup_iters=200, tau_max=0.99)
        zs = variational_z.sample_q_z(torch.tensor(ys_train).to('cuda').float().unsqueeze(0).expand(20, -1, -1), 0.99)
        zs2 = variational_z.sample_q_z(torch.tensor(ys_test).to('cuda').float().unsqueeze(0).expand(20, -1, -1), 0.99)
        pred_ys2=model.smooth(torch.tensor(ys_test).to('cuda').float(), zs2).mean(dim=0).detach().cpu().numpy() # [T, N]
        print(f"Test accuracy for {K} states:", train_metrics(ys_test, pred_ys2, None, k_max=0))
        Z_train = torch.mean(zs, dim=0).detach().cpu().numpy()
        Y_train = np.array(zs_train)
        Z_test = torch.mean(zs2, dim=0).detach().cpu().numpy()
        Y_test = np.array(zs_test)
        # Train KNN classifier
        knn = KNeighborsClassifier(n_neighbors=10, metric='euclidean')
        knn.fit(Z_train, Y_train)
        acc = knn.score(Z_test, Y_test)
        print(f"State quality for {K} states:", acc)
        accs.append(acc)  
    print("Finish one round,", accs)
    gsd.append(np.array(accs))
    np.save('./saved2024/f12024GSD.npy', np.array(gsd))

ELBO: -92030.3, LR: 0.01000, Tau: 0.990:   5%|▍         | 193/4000 [00:02<00:47, 80.09it/s] /opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
ELBO: 171215.2, LR: 0.00100, Tau: 0.990: 100%|██████████| 4000/4000 [00:50<00:00, 79.85it/s]


Test accuracy for 2 states: [0.9849207501261057]
State quality for 2 states: 0.3610425006387872


ELBO: 30954.1, LR: 0.01000, Tau: 0.990:   5%|▍         | 192/4000 [00:02<00:52, 72.40it/s]  /opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
ELBO: 494214.8, LR: 0.00100, Tau: 0.990: 100%|██████████| 4000/4000 [00:55<00:00, 72.24it/s]


Test accuracy for 3 states: [0.9991589784671543]
State quality for 3 states: 0.6589302444425518


ELBO: -29018.9, LR: 0.01000, Tau: 0.990:   5%|▍         | 196/4000 [00:03<00:59, 63.77it/s] /opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
ELBO: 470370.8, LR: 0.00100, Tau: 0.990: 100%|██████████| 4000/4000 [01:03<00:00, 63.48it/s]


...

ELBO: -163589.0, LR: 0.01000, Tau: 0.990:   5%|▍         | 196/4000 [00:03<00:59, 63.89it/s]/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
ELBO: 4208.2, LR: 0.00100, Tau: 0.990:  70%|███████   | 2813/4000 [00:44<00:18, 63.37it/s]  


Early stopping at iteration 2813.
Test accuracy for 4 states: [0.9709867444757606]
State quality for 4 states: 0.42619879056298443


ELBO: -55873.4, LR: 0.01000, Tau: 0.990:   5%|▍         | 198/4000 [00:03<01:08, 55.19it/s] /opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
ELBO: 424680.3, LR: 0.00100, Tau: 0.990: 100%|██████████| 4000/4000 [01:10<00:00, 56.75it/s]


Test accuracy for 5 states: [0.9984357895503919]
State quality for 5 states: 0.4857763393237373


ELBO: -61045.7, LR: 0.01000, Tau: 0.990:   5%|▍         | 198/4000 [00:03<01:09, 54.42it/s] /opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
ELBO: 184472.9, LR: 0.00100, Tau: 0.990:  55%|█████▌    | 2208/4000 [00:40<00:32, 54.48it/s]

## rSLDS

In [6]:
import ssm
from ssm.extensions.mp_srslds.transitions_ext import StickyRecurrentOnlyTransitions, StickyRecurrentTransitions

In [7]:
slds_acc = []
slds_sil = []
for seed in range(3):  ## repeat tests
    torch.manual_seed(seed)  # Set PyTorch seed
    torch.cuda.manual_seed_all(seed)  # Set CUDA seed 
    accs2 = []
    sils = []
    for K in range(2, 11):
        sro_trans=StickyRecurrentOnlyTransitions(K=K,D=2) 
        rslds = ssm.SLDS(N=10, K=K, D=2,
                 transitions=sro_trans,
                 dynamics="diagonal_gaussian",
                 emissions="gaussian",
                 single_subspace=True)
        rslds.initialize(ys_train)
        q_elbos_lem, q_lem = rslds.fit(ys_train, method="laplace_em", 
                                   variational_posterior="structured_meanfield",
                                   initialize=False, num_iters=100, alpha=0.0)
        q_lem_x = q_lem.mean_continuous_states[0]
        q_lem_z = rslds.most_likely_states(q_lem_x, ys_train)
        # plot_trajectory(q_lem_z, q_lem_x)
        elbos, q_lem2 = rslds.approximate_posterior(ys_test,
                                        method="laplace_em",
                                        variational_posterior="structured_meanfield",
                                        num_iters=20)
        q_lem_x2 = q_lem2.mean_continuous_states[0]
        q_lem_z2 = rslds.most_likely_states(q_lem_x2, ys_test)
        sil = silhouette_score(q_lem_x2, q_lem_z2)
        pred_ys2=rslds.smooth(q_lem_x2, ys_test)
        print(f"Test accuracy for {K} states:", train_metrics(ys_test, pred_ys2, None, k_max=0))
        print(f"Silhouette score at {K} states:", sil)
        Z_train = np.eye(K)[q_lem_z]
        Y_train = np.array(zs_train)
        Z_test = np.eye(K)[q_lem_z2]
        Y_test = np.array(zs_test)
        # Train KNN classifier
        knn = KNeighborsClassifier(n_neighbors=10, metric='euclidean')
        knn.fit(Z_train, Y_train)
        acc = knn.score(Z_test, Y_test)
        print(f"State quality for {K} states:", acc)
        accs2.append(acc)
        sils.append(sil)
    print("Finish one round,", accs2)
    slds_acc.append(np.array(accs2))
    slds_sil.append(np.array(sils))
    np.save('./saved2024/f12024SLDS.npy', np.array(slds_acc))    
    np.save('./saved2024/f12024SLDS_sil.npy', np.array(slds_sil))   

ELBO: 651184.6: 100%|██████████| 20/20 [00:23<00:00,  1.19s/it]


Test accuracy for 2 states: [0.9997806517054715]
Silhouette score at 2 states: 0.043377017763064435
State quality for 2 states: 0.03560173750106464


ELBO: 660121.1: 100%|██████████| 20/20 [00:26<00:00,  1.34s/it]


Test accuracy for 3 states: [0.9997805796289772]
Silhouette score at 3 states: -0.042072581759947034
State quality for 3 states: 0.15910058768418364


ELBO: 665660.7: 100%|██████████| 20/20 [00:32<00:00,  1.62s/it]


...

Test accuracy for 8 states: [0.999779133741623]
Silhouette score at 8 states: -0.08528136025250942
State quality for 8 states: 0.23294438293160719


ELBO: 673013.0: 100%|██████████| 20/20 [01:00<00:00,  3.04s/it]


Test accuracy for 9 states: [0.9997784174535924]
Silhouette score at 9 states: -0.15391312369341537
State quality for 9 states: 0.2769781108934503


ELBO: 673192.6: 100%|██████████| 20/20 [01:09<00:00,  3.49s/it]


Test accuracy for 10 states: [0.9997786277510159]
Silhouette score at 10 states: -0.13996371548986072
State quality for 10 states: 0.24614598415807853
Finish one round, [0.121965761008432, 0.1544587343497147, 0.21948726684268802, 0.1399369730005962, 0.2188910654969764, 0.2511711097862192, 0.23294438293160719, 0.2769781108934503, 0.24614598415807853]
